In [1]:
import matplotlib
matplotlib.use('SVG')
# cd to base directory of this repository to enable the loading of NEURON simulations
import os
os.chdir('/home/david/Projects/optogenetic_whole_cell_activation/morph_impact_on_opt_stim')
import numpy as np
import pandas as pd
from neuron import h
from neurostim.cell import Cell
from neurostim.light_classes import LightSource, LightStimulation
from neurostim.utils import convert_polar_to_cartesian_xz, interpolate
from neurostim.polarmaps import get_AP_times
import matplotlib.pyplot as plt

### Model Configuration
* cell type (loaded from ".hoc" file)
    * L5: 'L5.hoc' or L23: 'L23.hoc'
* ChR2 distribution inside cell
    * uniform: 'uniform'
    * soft soma targeting: 'shemesh_supfig9b_exp_yoff'
    * strict soma targeting: 'shemesh_supfig9b_exp_lin_yoff'
* ChR2 expression level (number ChR2 molecules in neuron's membrane)
    * typical number for L5 size (Foutz et al 2012): 10354945, is automatically scaled if 'L23.hoc' is selected
* cell depth in cortex (distance cortical surface - soma), [um]
    * {'L23': 400, 'L5': 1170}
* light model
    * currently only feasible model is for optical fiber [Foutz et al 2012]: 'foutz_et_al2012'
* optical fiber diameter
    * e.g. 25, 50, 100, 200, 400, um
* numerical aperture of fiber (changes beam divergence)
    * e.g. 0.1, 0.22, 0.39, 0.5
* light power [mW]
    * e.g. 1e-5 to 1e-2
* light duration [ms]
    * e.g. 30ms or 200ms
* position of light source on the cortical surface relative to soma of the neuron
    * radial distance: radius [um]
    * angle

## other parameters (meta params for simulation, analysis params etc.)
* total recording time of simulation
* AP threshold: threshold defining an action potential: 0mV
* light delay: 1ms (stim onset after simulation start)
* interpol_dt: 0.1ms (interpolation time for simulation results as simulation has adaptive steps)

In [2]:
# model config 1 ---- *** DO NOT RERUN ***
# NEURON simulator setup is corrupted if reinitialized in a running kernel, rerun complete notebook

# neuron model
hoc_file = 'L5'
# cortical depth of neuron models
cortical_depth = {'L23': 400, 'L5': 1170} #um

# ChR2 expression distribution and level
chanrhod_distribution = 'uniform' 
chanrhod_soma_density = 13e9 # 1/cm2

# NEURON setup
h.load_file("stdrun.hoc")
h.cvode_active(1)
# load cell and create stimulation object
cell = Cell(
    hoc_file="simneurostim/model/hoc/" + str(hoc_file) + ".hoc",
    cortical_depth=cortical_depth,
    ChR_soma_density=float(chanrhod_soma_density),
    ChR_distribution=str(chanrhod_distribution),
    rm_mech_from_secs=None,
    delete_all_secs_except_soma=False)

	1 
	1 
	1 
	1 


In [3]:
# model config 2 (can be adapted and rerun without restarting notebook kernel)

def simulate(light_power, light_duration=250, radius=0, angle=0, fiber_NA=0.22, fiber_diameter=200):

    # light source model 
    light_model = 'foutz_et_al2012'

    # other params
    light_delay = 290 #ms
    tot_rec_time = 800
    AP_threshold = 0 # mV
    interpol_dt = 0.1 # ms

    # light source initialization
    light_x, light_y = convert_polar_to_cartesian_xz(radius, angle)
    light_z = 0  # cortical surface
    light_pos = (light_x, light_y, light_z)
    light_source = LightSource(
        model=str(light_model),
        position=(light_x, light_y, light_z),
        width=float(fiber_diameter),
        NA=float(fiber_NA)
    )
    # stimulation object initialization
    light_stim = LightStimulation(
        cell=cell,
        light_source=light_source,
        delay=float(light_delay),
        duration=float(light_duration),
        light_power=float(light_power),
        record_all_segments=False,
    )
    # simulate
    measurement = pd.DataFrame(
        light_stim.simulate_and_measure(
            tot_rec_time=float(tot_rec_time),
            extra_rec_var_names=['Ina','Ik','Ica','I_chanrhod','gdens1_chanrhod'],
            extra_rec_var_pointers=[h.soma(0.5)._ref_ina, h.soma(0.5)._ref_ik, h.soma(0.5)._ref_ica, 
                                    h.soma(0.5)._ref_icat_chanrhod, h.soma(0.5)._ref_gdens1_chanrhod]
        )
    )
    # dealing with drop full row duplicates
    # drop completely redundant duplicates
    measurement = measurement.drop_duplicates()
    # add 1e-12 ms to 2nd entry time point of duplicate entries with the same time but different (e.g. Vm) values
    measurement.loc[measurement["time [ms]"].diff() == 0, "time [ms]"] += 1e-12
    # interpolate simulation results
    measurement = interpolate(
        df=measurement, interpolation_dt=float(interpol_dt)
    )
    # extract spike times
    AP_times = get_AP_times(
        df=measurement,
        interpol_dt=float(interpol_dt),
        t_on=float(light_delay),
        AP_threshold=AP_threshold
    )
    return measurement, AP_times, light_delay, light_duration

def simulate_and_plot(light_power, light_duration=200, radius=0, angle=0, fiber_NA=0.22, fiber_diameter=200):

    measurement, AP_times = simulate(
        light_power, light_duration, 
        radius, angle, 
        fiber_NA, fiber_diameter)
    
    def plot_membrane_voltage_currents_spikes(measurement, xlim, ylim):
        fig, axs = plt.subplots(2,1,figsize=(15,8))
        axs[0].plot(measurement['time [ms]'],measurement['V_soma(0.5)'], label='soma measurement')
        axs[0].set_ylabel('V_m [mV]')
        axs[0].axhline(-71,label='rest', color='tab:red')
        #axs[0].axhline(-90,label='equilibrium - potassium (given in L5.hoc)', color='tab:green')
        axs[0].legend()
        axs[0].set_ylim(-80,50)

        for x in get_AP_times(
                df=measurement,
                interpol_dt=float(interpol_dt),
                t_on=float(light_delay),
                AP_threshold=AP_threshold
            ):
            axs[1].axvline(x, color='tab:brown')
        axs[1].plot(measurement['time [ms]'],measurement['Ina'], label='Ina')
        axs[1].plot(measurement['time [ms]'],measurement['Ik'], label='Ik')
        axs[1].plot(measurement['time [ms]'],measurement['Ica'], label='Ica')
        axs[1].plot(measurement['time [ms]'],measurement['I_chanrhod'], label='I_chanrhod')
        axs[1].set_xlabel('time [ms]')
        axs[1].set_ylabel('current mA/cm2')
        [ax.set_xlim(*xlim) for ax in axs]
        axs[1].set_ylim(*ylim)
        axs[1].legend(loc='center right')
        return fig, axs
    fig, axs = plot_membrane_voltage_currents_spikes(measurement, xlim=[0,250], ylim=[-0.4,0.4])
    axs[0].set_title('elicited spikes (>0mV): '+str(len(AP_times)))
    plt.show()

    # light power at position in W/cm2
    print('light intensity [W/cm2]')
    print('-----------------------')
    print('soma: 1000 µm: ' ,np.round(light_source.calculate_Tx_at_pos([h.soma(0.5).x_chanrhod, h.soma(0.5).y_chanrhod, h.soma(0.5).z_chanrhod]),3))

    print('apical 100 µm: ' ,np.round(light_source.calculate_Tx_at_pos([h.soma(0.5).x_chanrhod, h.soma(0.5).y_chanrhod, 100]),3))

    print('average at fiber output: ', light_power/(np.pi*(fiber_diameter*1e-4/2)**2))
    return measurement

In [18]:
#photons/cm2/s
# wavelength 450
# 8 x 10**16


# light intensity at level of the retina
def convert_photons_to_intensity(n_photons):
    h_planck = 6.62607015e-34 #Js
    lamda = 450e-9
    c = 3e8
    E_photon = h_planck * c/lamda
    return n_photons * E_photon

def convert_input_power_to_soma_photon_intensity(light_power):
    light_source = LightSource(
        model='foutz_et_al2012',
        position=(0, 0, 0),
        width=200,
        NA=0.22
    )
    input_power_to_soma_intensity_factor = light_source.calculate_Tx_at_pos(
        [0,0,h.soma(0.5).z_chanrhod])
    # photon energy:
    h_planck = 6.62607015e-34 #Js
    lamda = 450e-9
    c = 3e8
    E_photon = h_planck * c/lamda
    return light_power * input_power_to_soma_intensity_factor / E_photon
    

photon_intensities = [4e14, 2e16, 1e17, 3e17] # panel B
photon_intensities_cellE = [4e14, 2e16, 8e16, 1e17, 3e17] # panel D

stim_intensitiesA = [4.1e18, 1.4e18, 2.8e17, 5.5e15]
stim_intensitiesC = [7.6e18, 2.5e18, 2e18, 5.1e17, 1e16]

def print_factors(ints, cond, normidx):
    print(cond)
    if normidx==-1:
        print(np.array(ints)[::-1]/ints[normidx])
    else:
        print(np.array(ints)/ints[normidx])
print_factors(stim_intensitiesA, 'A', normidx=-1)    
print_factors(photon_intensities, 'B', normidx=0)
print_factors(stim_intensitiesC, 'C', normidx=-1)    
print_factors(photon_intensities_cellE, 'D', normidx=0)

A
[  1.          50.90909091 254.54545455 745.45454545]
B
[  1.  50. 250. 750.]
C
[  1.  51. 200. 250. 760.]
D
[  1.  50. 200. 250. 750.]


In [5]:
# dimensions (A4:  8.27 x 11.69 inches)
text_frac = 0.8
abs_pw = 8.27 # pagewidth
abs_pl = 11.69 # pagelength
pw = text_frac * abs_pw
pl = text_frac * abs_pl

# others
dpi=900

In [12]:
def run_simulation_for_k(k, name, photon_intensities):
    print('----------------------------------------')
    print(name)
    results = dict()
    simulated_light_powers = dict()
    for pi in photon_intensities:
        lp = k*convert_photons_to_intensity(pi)
        simulated_light_powers[pi] = lp
        measurement, AP_times, light_delay, light_duration = simulate(light_power=lp, light_duration=260)
        results[pi] = measurement
    l_ = len(photon_intensities)
    fig, axs = plt.subplots(l_,1,figsize=(pw/2,pw/2.5))
    for i, pi in enumerate(photon_intensities):
        measurement = results[pi]
        axs[l_-1-i].plot(measurement['time [ms]'],measurement['V_soma(0.5)'], 
                      label='soma measurement', color='black', lw=0.5)
        #axs[3-i].set_ylabel('V_m [mV]')
        axs[l_-1-i].set_ylim(-90,50)
        axs[l_-1-i].set_xlim(0,800)
        # Hide the right and top spines
        axs[l_-1-i].spines['right'].set_visible(False)
        axs[l_-1-i].spines['top'].set_visible(False)
        axs[l_-1-i].set_yticks([-70,0])
        axs[l_-1-i].set_yticks([-90,-80,-60,-50,-40,-30,-20,-10,10,20,30,40,50], minor=True)
        axs[l_-1-i].text(0.1, 0.6, '{:.1E}'.format(
            convert_input_power_to_soma_photon_intensity(simulated_light_powers[pi])
        ), transform=axs[l_-1-i].transAxes)
    for i in range(l_-1):
        axs[i].set_xticklabels([])
        axs[i].axvline(x=light_delay)
        axs[i].axvline(x=light_delay+light_duration)

    fig.savefig('paper/NEW_FIGURES/FIG_depoblock/simtraces_'+hoc_file+str(k)+'.svg', dpi=dpi, 
                facecolor=None, edgecolor=None,  
                bbox_inches='tight')
    plt.show()
# if hoc_file == 'L23':
#     for k in [1,1/3,1/9, 1/81,1/160,1/250]:
#         run_simulation_for_k(k, name='Cell')
# if hoc_file == 'L5':
#     for k in [1,1/3,1/9, 1/81,1/160,1/250]:
#         run_simulation_for_k(k, name='Cell')

In [7]:
photon_intensities

[400000000000000.0, 2e+16, 1e+17, 3e+17]

In [13]:
# Run simulation with L5 cell
# Antoine's cells: A (top), B,C,D (bottom left, from top to bottom), E (bottom right)
# Cell A k=1/3
run_simulation_for_k(0.3, name='Cell A', photon_intensities=photon_intensities)
# Cell B
#run_simulation_for_k(1/3, name='Cell B')
# Cell C
#run_simulation_for_k(1/3, name='Cell B')
# Cell D
#run_simulation_for_k(1/3, name='Cell B')
# Cell E
run_simulation_for_k(0.55, name='Cell E', photon_intensities=photon_intensities_cellE)

----------------------------------------
Cell E


/home/david/Software/miniconda3/envs/mioos/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Matplotlib is currently using svg, which is a non-GUI backend, so cannot show the figure.


In [9]:
#run_simulation_for_k(1/3, name='Cell A', photon_intensities=photon_intensities)

In [10]:
# calculate factors by which  stimulation itnensity increases
stim_intensitiesA = [4.1e18, 1.4e18, 2.8e17, 5.5e15]
stim_intensitiesB = [7.6e18, 2.5e18, 2e18, 5.1e17, 1e16]

15.333333333333334

In [11]:
for pi in photon_intensities:
    print('{:.5E}'.format(pi*k*46))
print('conversion factor: ', k*46)    

NameError: name 'k' is not defined